In [2]:
import numpy as np
import yfinance as yf
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
import IchimokuLibrary
from itertools import product
import arch.bootstrap as ab

In [3]:
START = "2015-01-01"
END = "2025-01-01"
TICKER = "^GSPC"
TITLE = TICKER + " - " + (datetime.now()).strftime("%Y-%m-%d - %H:%M:%S")
file = "confidence_intervals_sharpe.xlsx"

In [ ]:
trading_lengths = {
    'short': range(5,13,2),
    'medium': range(17,33,2),
    'long': range(40,60,2)
}

In [ ]:
columns = []
for strategy in IchimokuLibrary.STRATEGIES_L:
    ex = strategy[0]
    en_list = strategy[1]
    for en in en_list:
        for measure in ["mu", "sigma", "Sharpe"]:
            columns.append((ex, en, measure))

multi_index_columns = pd.MultiIndex.from_tuples(columns)
multi_index_index = pd.MultiIndex.from_tuples(product(trading_lengths['short'], trading_lengths['medium'],trading_lengths["long"], ["Lower", "Upper"]))

In [153]:
data = pd.read_excel("btc_usd_confidence_intervals_sharpe.xlsx",header=[0,1,2], index_col=[0,1,2,3])

In [154]:
# data = pd.read_excel("confidence_intervals_sharpe.xlsx",header=[0,1,2], index_col=[0,1,2,3])

In [155]:
(data.xs("Sharpe",axis=1, level=2).xs("Lower",axis=0,level=3)).shape

(600, 13)

In [156]:
(data.xs("Sharpe", level=2, axis=1).idxmax().values)

array([(7, 31, 35, 'Upper'), (7, 31, 35, 'Upper'), (7, 31, 35, 'Upper'),
       (5, 27, 39, 'Upper'), (9, 17, 35, 'Upper'), (13, 25, 35, 'Upper'),
       (13, 25, 47, 'Upper'), (5, 19, 35, 'Upper'), (7, 31, 35, 'Upper'),
       (7, 31, 37, 'Upper'), (13, 25, 35, 'Upper'), (13, 25, 41, 'Upper'),
       (7, 25, 53, 'Upper')], dtype=object)

In [158]:
sharpes = ((data.xs("Sharpe", level=2, axis=1).xs("Upper", level=3, axis=0) + data.xs("Sharpe", level=2, axis=1).xs("Lower", level=3, axis=0)) / 2)

In [163]:
sharpes.idxmax() #* np.sqrt(252)

ex_l_1  en_l_1       (7, 31, 35)
        en_l_2       (7, 31, 35)
        en_l_12      (7, 31, 35)
        en_l_123     (5, 27, 37)
ex_l_2  en_l_1      (13, 31, 35)
        en_l_12     (13, 31, 35)
        en_l_123    (13, 31, 45)
ex_l_3  en_l_2       (9, 21, 35)
        en_l_12      (7, 31, 35)
        en_l_123     (7, 31, 37)
ex_l_4  en_l_12     (13, 31, 35)
        en_l_123    (13, 25, 37)
ex_l_5  en_l_123     (5, 25, 35)
dtype: object

In [169]:
(sharpes.stack([0,1]).nlargest(100).sort_values(ascending=False) * np.sqrt(252)).reset_index()

C:\Users\olive\AppData\Local\Temp\ipykernel_13984\653513569.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  (sharpes.stack([0,1]).nlargest(100).sort_values(ascending=False) * np.sqrt(252)).reset_index()


,level_0,level_1,level_2,level_3,level_4,0
0,5,25,35,ex_l_5,en_l_123,1.179554
1,5,25,37,ex_l_5,en_l_123,1.169487
2,5,25,39,ex_l_5,en_l_123,1.166983
3,5,25,41,ex_l_5,en_l_123,1.164581
4,5,25,43,ex_l_5,en_l_123,1.161644
...,...,...,...,...,...,...
95,9,23,43,ex_l_5,en_l_123,1.088042
96,5,21,59,ex_l_5,en_l_123,1.088010
97,13,25,39,ex_l_5,en_l_123,1.087971
98,11,27,41,ex_l_5,en_l_123,1.086395
